# Module 1.8: Tool Calling Agent with LangGraph

**Duration:** 60 minutes  
**Objective:** Build a real ReAct agent with tool calling capabilities

In this exercise, you'll learn:
- Defining tools using `@tool` decorator
- Binding tools to LLM with `bind_tools()`
- Using `ToolNode` for automatic tool execution
- Implementing `tools_condition` for routing
- Building a complete ReAct loop

<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agents-course-2025/blob/main/exercises/langgraph/1.3-langgraph-tool-calling.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

## Setup

### Install dependencies

In [ ]:
%pip install openai~=1.57 --upgrade --quiet
%pip install python-dotenv~=1.0 --upgrade --quiet
%pip install langchain~=0.3 langchain_openai~=0.2 --upgrade --quiet
%pip install langgraph~=0.2 --upgrade --quiet

### Load environment variables

In [ ]:
import os

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    # Get API key from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    # Load from .env file for local development
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed. Install with: pip install python-dotenv")

# Verify API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar")
        print("   → Add a secret named 'OPENAI_API_KEY'")
        print("   → Toggle 'Notebook access' to enable it")
    else:
        print("   → Create a .env file with: OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key configured!")

### Setup LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

---

## Part 1: Define Tools (15 min)

### What are Tools?

Tools are functions that an agent can call to:
- Access external data (APIs, databases)
- Perform calculations
- Execute actions (send email, create file)
- Interact with systems

### The `@tool` Decorator

LangChain provides a `@tool` decorator that:
1. Converts a Python function into a tool
2. Generates a JSON schema from docstring and type hints
3. Makes the tool compatible with LLM function calling

In [ ]:
from langchain_core.tools import tool
from typing import Literal
import random

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.
    
    Args:
        city: The name of the city to get weather for
        
    Returns:
        A string describing the current weather
    """
    print(f"🌤️  Calling weather API for: {city}")
    
    # Simulated weather data
    weather_data = {
        "stockholm": "5°C, cloudy with a chance of meatballs",
        "london": "12°C, rainy (as usual)",
        "tokyo": "18°C, clear skies",
        "new york": "8°C, windy",
        "sydney": "25°C, sunny"
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"The weather in {city} is: {weather_data[city_lower]}"
    else:
        return f"Weather data not available for {city}. Try Stockholm, London, Tokyo, New York, or Sydney."


@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 2", "10 * 5")
        
    Returns:
        The result of the calculation as a string
    """
    print(f"🔢 Calculating: {expression}")
    
    try:
        # Safe evaluation (restricted to basic math)
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"


@tool
def get_random_fact(topic: Literal["space", "ocean", "animals", "history"]) -> str:
    """Get a random interesting fact about a topic.
    
    Args:
        topic: The topic to get a fact about (space, ocean, animals, or history)
        
    Returns:
        An interesting fact as a string
    """
    print(f"📚 Fetching random fact about: {topic}")
    
    facts = {
        "space": [
            "A day on Venus is longer than a year on Venus.",
            "Neutron stars are so dense that a teaspoon would weigh billions of tons.",
            "There are more stars in the universe than grains of sand on all Earth's beaches."
        ],
        "ocean": [
            "The ocean produces more than 50% of the world's oxygen.",
            "The deepest part of the ocean is nearly 7 miles down.",
            "More people have been to the moon than to the deepest part of the ocean."
        ],
        "animals": [
            "Octopuses have three hearts and blue blood.",
            "A group of flamingos is called a 'flamboyance'.",
            "Honey never spoils. Archaeologists have found 3000-year-old honey that's still edible."
        ],
        "history": [
            "Cleopatra lived closer to the Moon landing than to the building of the pyramids.",
            "Oxford University is older than the Aztec Empire.",
            "The first programmer was Ada Lovelace in the 1840s."
        ]
    }
    
    return random.choice(facts[topic])

### Test Tools Independently

In [ ]:
# Test weather tool
print(get_weather.invoke("Stockholm"))
print()

# Test calculator
print(calculate.invoke("42 * 17"))
print()

# Test fact tool
print(get_random_fact.invoke("space"))

### Inspect Tool Schema

See how LangChain converts your function into a tool schema:

In [ ]:
import json

print("Weather Tool Schema:")
print(json.dumps(get_weather.args_schema.schema(), indent=2))

---

## Part 2: Build Tool-Calling Graph (30 min)

### The ReAct Pattern

**ReAct** = **Rea**soning + **Act**ing

The agent follows this loop:
1. **Think**: Decide what to do based on the question
2. **Act**: Call a tool if needed
3. **Observe**: See the tool's result
4. **Repeat** or **Answer**: Continue until done

### Bind Tools to LLM

First, we "bind" our tools to the LLM, which tells it what tools are available:

In [ ]:
# Collect all tools
tools = [get_weather, calculate, get_random_fact]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

print(f"✅ Bound {len(tools)} tools to LLM")
print(f"   Tools: {[t.name for t in tools]}")

### Define Assistant Node

The assistant node calls the LLM with tools. It uses **MessagesState** which includes conversation history.

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage

# System message defines the assistant's behavior
system_message = SystemMessage(
    content="""You are a helpful assistant with access to tools.
    
    When users ask questions:
    1. Think about what information you need
    2. Use the appropriate tool to get that information
    3. Provide a helpful, friendly answer based on the tool results
    
    Available tools:
    - get_weather: Get current weather for a city
    - calculate: Evaluate mathematical expressions
    - get_random_fact: Get interesting facts about topics
    
    Always use tools when you need information. Don't make up data.
    """
)

def assistant_node(state: MessagesState) -> MessagesState:
    """The main assistant node that calls LLM with tools."""
    print("\n🤖 Assistant thinking...")
    
    # Call LLM with tools
    response = llm_with_tools.invoke([system_message] + state["messages"])
    
    # Check if tool calls were made
    if response.tool_calls:
        print(f"   📞 Calling {len(response.tool_calls)} tool(s)")
        for tc in response.tool_calls:
            print(f"      - {tc['name']}({tc['args']})")
    else:
        print("   💬 Responding to user (no tools needed)")
    
    return {"messages": [response]}

### Use ToolNode for Automatic Execution

`ToolNode` automatically:
1. Extracts tool calls from the LLM response
2. Executes the appropriate tool functions
3. Returns the results as ToolMessages

In [ ]:
from langgraph.prebuilt import ToolNode

# Create tool node with our tools
tool_node = ToolNode(tools)

print("✅ Created ToolNode for automatic tool execution")

### Build the Graph with tools_condition

`tools_condition` is a built-in conditional function that:
- Routes to "tools" if the LLM made tool calls
- Routes to END if the LLM provided a final answer

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display

# Build the graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

# Add edges
builder.add_edge(START, "assistant")

# Conditional edge: if tool calls exist, go to tools; otherwise END
builder.add_conditional_edges(
    "assistant",
    tools_condition,  # Built-in function that checks for tool calls
)

# After tools execute, loop back to assistant
builder.add_edge("tools", "assistant")

# Compile
graph = builder.compile()

# Visualize - note the xray=True to see internal structure
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

---

## Part 3: Test and Debug (15 min)

### Single Tool Call Test

In [ ]:
from langchain_core.messages import HumanMessage

# Simple question requiring one tool
question = "What's the weather like in Stockholm?"

print(f"❓ Question: {question}")
print("="*80)

result = graph.invoke({
    "messages": [HumanMessage(content=question)]
})

# Print the conversation
print("\n💬 Conversation:")
for msg in result["messages"]:
    msg.pretty_print()

### Multiple Tool Calls Test

In [ ]:
# Complex question requiring multiple tools
question = "Calculate 123 * 456 and also tell me the weather in Tokyo"

print(f"❓ Question: {question}")
print("="*80)

result = graph.invoke({
    "messages": [HumanMessage(content=question)]
})

print("\n💬 Conversation:")
for msg in result["messages"]:
    msg.pretty_print()

### Chain of Tool Calls Test

Test if the agent can use tool results to make follow-up tool calls:

In [ ]:
# Question that requires sequential reasoning
question = """Calculate 15 * 8, then tell me if that number is greater than 100. 
If it is, give me a space fact. If not, give me an ocean fact."""

print(f"❓ Question: {question}")
print("="*80)

result = graph.invoke({
    "messages": [HumanMessage(content=question)]
})

print("\n💬 Final Answer:")
result["messages"][-1].pretty_print()

### Stream the Execution

See the agent's thought process in real-time:

In [ ]:
question = "What's the weather in London and give me a history fact"

print(f"❓ Question: {question}")
print("="*80)
print("\n🔄 Streaming execution:\n")

for step in graph.stream({"messages": [HumanMessage(content=question)]}):
    print(f"Step: {list(step.keys())[0]}")
    print(f"  Content: {step}")
    print()

---

## 🎯 Exercise: Build a Research Assistant Agent

**Challenge:** Create a research assistant with more advanced tools.

### Requirements:

1. **Create these tools**:
   - `search_wikipedia(topic: str)` - Simulate Wikipedia search
   - `get_current_date()` - Return today's date
   - `convert_currency(amount: float, from_currency: str, to_currency: str)` - Currency conversion

2. **Build a ReAct agent** that can:
   - Answer questions using multiple tools
   - Handle tool failures gracefully
   - Provide sourced, accurate answers

3. **Test with complex queries**:
   - "What year was Python created and how many years ago was that?"
   - "Convert 100 USD to EUR and then to JPY"

### Template

In [ ]:
from datetime import datetime

# TOOLS
@tool
def search_wikipedia(topic: str) -> str:
    """Search Wikipedia for information about a topic.
    
    Args:
        topic: The topic to search for
    """
    # TODO: Implement simulated Wikipedia search
    # Create a dict with some topics and return relevant info
    wiki_data = {
        "python": "Python was created by Guido van Rossum and first released in 1991.",
        "langchain": "LangChain is a framework for developing LLM applications, created in 2022.",
        # Add more topics...
    }
    return wiki_data.get(topic.lower(), f"No information found for {topic}")

@tool
def get_current_date() -> str:
    """Get the current date."""
    # TODO: Return current date in readable format
    return datetime.now().strftime("%Y-%m-%d")

@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert currency from one type to another.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency (USD, EUR, JPY, etc.)
        to_currency: Target currency (USD, EUR, JPY, etc.)
    """
    # TODO: Implement simulated currency conversion
    # Use approximate exchange rates
    rates = {
        ("USD", "EUR"): 0.92,
        ("EUR", "USD"): 1.09,
        ("USD", "JPY"): 149.50,
        # Add more rates...
    }
    
    pair = (from_currency.upper(), to_currency.upper())
    if pair in rates:
        result = amount * rates[pair]
        return f"{amount} {from_currency} = {result:.2f} {to_currency}"
    return f"Conversion rate not available for {from_currency} to {to_currency}"

# TODO: Build the agent graph with these tools
# research_tools = [search_wikipedia, get_current_date, convert_currency]
# ...

# TODO: Test with complex queries

---

## Key Takeaways

✅ **Tool Definition**: Use `@tool` decorator for automatic schema generation  
✅ **Tool Binding**: `bind_tools()` tells LLM what tools are available  
✅ **ToolNode**: Automatically executes tools based on LLM output  
✅ **tools_condition**: Built-in routing for tool-calling loops  
✅ **ReAct Pattern**: Think → Act → Observe → Repeat  
✅ **MessagesState**: Maintains conversation history automatically  

### Best Practices for Tool-Calling Agents

**Tool Design:**
- Clear, descriptive docstrings (LLM reads these!)
- Type hints for all parameters
- Meaningful parameter names
- Handle errors gracefully
- Return strings (easiest for LLM to process)

**Agent Design:**
- Start with system message defining behavior
- Use `parallel_tool_calls=False` for debugging
- Add logging/printing to understand tool calls
- Test tools independently first
- Handle tool failures in the system prompt

**Debugging:**
- Stream execution to see each step
- Check tool schemas with `.args_schema.schema()`
- Use `xray=True` in graph visualization
- Test with increasingly complex queries

### Common Pitfalls

❌ **Poor docstrings** → LLM doesn't know when to use tool  
❌ **Complex return types** → LLM can't interpret results  
❌ **No error handling** → Agent gets stuck on failures  
❌ **Too many tools** → LLM gets confused about which to use  
❌ **Vague tool names** → LLM calls wrong tool  

### Next Steps

- Day 2: RAG with LangGraph (Module 2.4)
- Learn how to combine tools with retrieval
- Build production-ready agents

---

## Additional Resources

- [LangGraph Tool Calling Guide](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/)
- [LangChain Tools Documentation](https://python.langchain.com/docs/concepts/tools/)
- [ToolNode API Reference](https://langchain-ai.github.io/langgraph/reference/prebuilt/#toolnode)
- [ReAct Paper (2022)](https://arxiv.org/abs/2210.03629)